In [3]:
import geopandas as gpd
import pandas as pd

# Cargar las rutas desde el archivo SHP
gdf_rutas = gpd.read_file(r"C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos Rutas\archivo_atributos_unificados.shp")

# Cargar las mallas de la Península y de Canarias
gdf_malla_peninsula = gpd.read_file(r"C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos IEET\Mallas Península\Malla10km_p.shp")
gdf_malla_canarias = gpd.read_file(r"C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos IEET\Mallas Canarias\Malla10km_c.shp")

# Cargar el archivo CSV de las rutas
df_rutas_csv = pd.read_csv(r"C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos Rutas\archivo_rutas.csv")


In [4]:
# Asegurarse de que ambos GeoDataFrames usan el mismo CRS
gdf_rutas = gdf_rutas.to_crs("EPSG:4326")
gdf_malla_peninsula = gdf_malla_peninsula.to_crs("EPSG:4326")
gdf_malla_canarias = gdf_malla_canarias.to_crs("EPSG:4326")

In [ ]:
# Unir las rutas con la malla de la Península
gdf_rutas_peninsula = gpd.sjoin(gdf_rutas, gdf_malla_peninsula, how="left", predicate="intersects")

# Unir las rutas con la malla de Canarias
gdf_rutas_canarias = gpd.sjoin(gdf_rutas, gdf_malla_canarias, how="left", predicate="intersects")


# Unir los resultados de las rutas en Península y Canarias
gdf_rutas_combinadas = pd.concat([gdf_rutas_peninsula, gdf_rutas_canarias], ignore_index=True)

# Eliminar filas duplicadas (en caso de que una ruta esté en ambas mallas)
gdf_rutas_combinadas = gdf_rutas_combinadas.drop_duplicates(subset='id_ruta')  # Usa el identificador único de cada ruta

# Ver las primeras filas para comprobar el resultado
print(gdf_rutas_combinadas[['id_ruta', 'codigo_malla']].head())


In [ ]:
# Unir el DataFrame original de rutas con el GeoDataFrame combinado
df_rutas_con_malla = pd.merge(df_rutas_csv, gdf_rutas_combinadas[['id_ruta', 'codigo_malla']], 
                               on='id_ruta', how='left')

# Verificar el resultado
print(df_rutas_con_malla.head())

In [ ]:
# Guardar el archivo CSV con la nueva columna
df_rutas_con_malla.to_csv("ruta_a_guardar_el_csv_con_malla.csv", index=False)